In [ ]:
pip install langchain_community

In [ ]:
pip install gpt4all

In [ ]:
pip install --upgrade langchain==0.0.353

In [2]:
pip list | grep langchain

langchain                     0.0.353
langchain-community           0.0.11
langchain-core                0.1.9
langchainhub                  0.1.14

[notice] A new release of pip is available: 23.0 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import langchain
import langchain_community
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain import vectorstores
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import GPT4All
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community import vectorstores

In [4]:
path_name = '/Users/andkelly/Library/CloudStorage/Dropbox/Applications Folder/Resumes'

In [5]:
loader = DirectoryLoader(path_name, glob="**/*.pdf")

In [6]:
docs = loader.load()

In [7]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=256, chunk_overlap=0
)

In [8]:
texts = text_splitter.split_documents(docs)

In [9]:
texts = langchain_community.vectorstores.utils.filter_complex_metadata(texts)

In [10]:
vectorstore = Chroma.from_documents(documents=texts, embedding=GPT4AllEmbeddings())

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [ ]:
question = "What has Andrew done?"

In [ ]:
docs = vectorstore.similarity_search(question)

In [ ]:
len(docs)

In [ ]:
docs[0]

In [11]:
llm = GPT4All(
    model="/Users/andkelly/Applications/models/mistral-7b-openorca.Q4_0.gguf",
    max_tokens=2048,
)

In [ ]:
output = gpt4all.generate(["The capital of the United States is..."], max_tokens=3)

In [ ]:
print(output)

In [ ]:
prompt = PromptTemplate.from_template(
    "Give me Andrew's elevator pitch from these retrieved docs: {docs}"
)

In [17]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
prompt

In [ ]:
chain = {"docs": format_docs} | prompt | llm | StrOutputParser()

In [ ]:
question = "Tell me about yourself"
docs = vectorstore.similarity_search(question)
chain.invoke(docs)

In [ ]:
pip install langchainhub

In [12]:
from langchain import hub

In [13]:
rag_prompt = hub.pull("rlm/rag-prompt")

In [ ]:
dict(rag_prompt.messages)

In [19]:
from langchain_core.runnables import RunnablePassthrough, RunnablePick

In [ ]:
RunnablePick("context")

In [ ]:
RunnablePassthrough.assign(context = RunnablePick("context") | format_docs)

In [ ]:
RunnablePassthrough.assign(context=RunnablePick("context") | format_docs) | rag_prompt.messages

In [ ]:
rag_prompt[0].prompt.map

In [ ]:
chain = (
    RunnablePassthrough.assign(context=RunnablePick("context") | format_docs)
    | rag_prompt[0].prompt.map
    | llm
    | StrOutputParser()
)

In [21]:
question = "Tell me about yourself"

In [ ]:
chain.invoke({"context": docs, "question": question})

In [15]:
retriever = vectorstore.as_retriever()

In [29]:
context = lambda x: format_docs(x["context"])

In [30]:
context

<function __main__.<lambda>(x)>

In [35]:
rag_chain_from_docs = (
    # {"context": retriever | format_docs, "question": RunnablePassthrough()}
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [33]:
from langchain_core.runnables import RunnableParallel

In [36]:
rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer = rag_chain_from_docs)

In [38]:
response = rag_chain_with_source.invoke("What is Andrew's elevator pitch?")

In [57]:
response['context'][0].metadata['source']

'/Users/andkelly/Library/CloudStorage/Dropbox/Applications Folder/Resumes/2023_Kelly_Andrew_Capital_One_Resume.pdf'

In [26]:
qa_chain.invoke(question)

AssertionError: The input to RunnablePassthrough.assign() must be a dict.

# Update Q&A Doc Retrieval w/ Source

In [ ]:
def make_retriever(path_name):
    loader = DirectoryLoader(path_name, glob="**/*.pdf")
    docs = loader.load()
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=256, chunk_overlap=0
    )
    texts = text_splitter.split_documents(docs)
    texts = langchain_community.vectorstores.utils.filter_complex_metadata(texts)
    embeddings = GPT4AllEmbeddings()
    vectorstore = Chroma.from_documents(texts, embeddings)
    retriever = vectorstore.as_retriever()
    return retriever

In [ ]:
def get_answer(user_input, chat_history, retriever):
    llm = GPT4All(
        model="/Users/andkelly/Applications/models/mistral-7b-openorca.Q4_0.gguf",
        max_tokens=2048
    )
    
    _template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
    
	Chat History:
	{chat_history}
	Follow Up Input: {question}
	Standalone question:"""
    
    CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)
    
    question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
    
    doc_chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")
    
    chain = ConversationalRetrievalChain(
        retriever=retriever,
        question_generator=question_generator,
        combine_docs_chain=doc_chain
    )
    
    result = chain({"question": user_input, "chat_history": chat_history})
    
    return result["answer"]

In [ ]:
print(CONDENSE_QUESTION_PROMPT)

In [ ]:
while True:
	path_name = input("Enter path name: ")
	retriever = make_retriever(path_name)
	print("Ask a question...")
	chat_history = []
	while True:
		user_input = input("You: ")
		response = get_answer(user_input, chat_history, retriever)
		chat_history = [(user_input, response)]
		print("Response: ", response)

Note: When I use a local model in the previously developed Q&A over docs that cites sources, I was returned an error about instantiating classess for the BasePromptTemplate and BaseLanguageModel. As a result, I'm going to try followiung the turtorial docs [here](https://python.langchain.com/docs/use_cases/question_answering/local_retrieval_qa#using-in-a-chain). If that fails, then I can explore the documentation [here](https://python.langchain.com/docs/use_cases/question_answering/quickstart). Last resort is to reach out to Alex Talbot to see what they did for CDER.